## Work flow/Structure:
### STEP1: Find the HSV range of the target Marker/pen and save the values in a .npy file
### STEP 2: Use the above values to start drawing in the main code
First, we will use color masking to get a mask of our colored pen/marker using the above HSV range.
Next, using contour detection we detect and track the location of that pen,i.e get the x,y coordinates.
Next, draw a line by joining the x,y coordinates of pen’s previous location (location in the previous frame) with the new x,y points.
Add a feature to use the marker as an Eraser to erase unwanted lines.
Finally, add another feature to clear the entire Canvas/Screen.

In [6]:
#finding hsv range of target object(pen)
import cv2
import numpy as np
import time
# A required callback method that goes into the trackbar function.
def nothing(x):
    pass

# Define a capture device .Here we use our primary laptop camera(0).If using external usb camera use replace 0 with 1.
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

# Create a window named trackbars.
cv2.namedWindow("Trackbars")

# Now create 6 trackbars that will control the lower and upper range of 
# H,S and V channels. The Arguments are like this: Name of trackbar, 
# window name, range,callback function. For Hue the range is 0-179 and
# for S,V its 0-255.
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)
 
while True:
    
    # Start reading the webcam feed frame by frame.
    ret, frame = cap.read()
    if not ret:
        break
    # Flip the frame horizontally (Not required)
    frame = cv2.flip( frame, 1 ) 
    
    # Convert the BGR image to HSV image.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Get the new values of the trackbar in real time as the user changes 
    # them
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")
 
    # Set the lower and upper HSV range according to the value selected
    # by the trackbar
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    
    # Filter the image and get the binary mask, where white represents 
    # your target color
    mask = cv2.inRange(hsv, lower_range, upper_range)
 
    # You can also visualize the real part of the target color (Optional)
    res = cv2.bitwise_and(frame, frame, mask=mask)
    
    # Converting the binary mask to 3 channel image, this is just so 
    # we can stack it with the others
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # stack the mask, orginal frame and the filtered result
    stacked = np.hstack((mask_3,frame,res))
    
    # Show this stacked frame at 40% of the size.
    cv2.imshow('Trackbars',cv2.resize(stacked,None,fx=0.4,fy=0.4))
    
    # If the user presses ESC then exit the program
    key = cv2.waitKey(1)
    if key == 27:
        break
    
    # If the user presses `s` then print this array.
    if key == ord('s'):
        
        thearray = [[l_h,l_s,l_v],[u_h, u_s, u_v]]
        print(thearray)
        
        # Also save this array as penval.npy
        np.save('hsv_value',thearray)
        break
    
# Release the camera & destroy the windows.    
cap.release()
cv2.destroyAllWindows()

[[111, 131, 108], [179, 255, 255]]


## Concepts Used
### Morphological Operations

### Basics of Erosion:

- Erodes away the boundaries of foreground object
- Used to diminish the features of an image.


### Working of erosion:


1. A kernel(a matrix of odd size(3,5,7) is convolved with the image.
2. A pixel in the original image (either 1 or 0) will be considered 1 only if all the pixels under the kernel is 1, otherwise it is eroded (made to zero).
3. Thus all the pixels near boundary will be discarded depending upon the size of kernel.
4. So the thickness or size of the foreground object decreases or simply white region decreases in the image.


### Basics of dilation:

- Increases the object area
- Used to make features more prominent.


### Working of dilation:

1. A kernel(a matrix of odd size(3,5,7) is convolved with the image
2. A pixel element in the original image is ‘1’ if atleast one pixel under the kernel is ‘1’.
3. It increases the white region in the image or size of foreground object increases

<img src="https://media.geeksforgeeks.org/wp-content/uploads/Selection_014.png">


### Practical Usecase
Erosion:
- It is useful for removing small white noises.
- Used to detach two connected objects etc.


Dilation:
- In cases like noise removal, erosion is followed by dilation. Because, erosion removes white noises, but it also shrinks our object. So we dilate it. Since noise is gone, they won’t come back, but our object area increases.
- It is also useful in joining broken parts of an object.

# Contours :
Contours are defined as the line joining all the points along the boundary of an image that are having the same intensity. Contours come handy in shape analysis, finding the size of the object of interest, and object detection.

### findContours() Method : Arguments
- First one is source image, second is contour retrieval mode, third is contour approximation method and it outputs the image, contours, and hierarchy. 
- ‘contours‘ is a Python list of all the contours in the image. 
- Each individual contour is a Numpy array of (x, y) coordinates of boundary points of the object.


## Hierarchy in Contour Detection

<img src="https://docs.opencv.org/master/hierarchy.png">

- 0,1,2 are external or outermost. We can say, they are in hierarchy-0 or simply they are in same hierarchy level.
- contour-2a can be considered as a child of contour-2 (hierarchy-1.)
- contour-3 is child of contour-2 and it comes in next hierarchy.
- contours 4,5 are the children of contour-3a, and they come in the last hierarchy.
- contour-4 is the first child of contour-3a (It can be contour-5 also).



### OpenCV represents it as an array of four values : [Next, Previous, First_Child, Parent]
1. Next denotes next contour at the same hierarchical level.

For eg, take contour-0 in our picture. Who is next contour in its same level ? It is contour-1. So simply put Next = 1. Similarly for Contour-1, next is contour-2. So Next = 2.
contour-2? There is no next contour in the same level. So simply, put Next = -1

2. Previous denotes previous contour at the same hierarchical level. 
For 0 contour it will be -1. For 2 it will be 1.

3. First_Child denotes its first child contour.

4. Parent denotes index of its parent contour.


### Contours Approximation Method :
- If we pass cv2.CHAIN_APPROX_NONE, all the boundary points are stored.
- cv2.CHAIN_APPROX_SIMPLE removes all redundant points and compresses the contour, thereby saving memory. (ex for a single line)

### Contour Retrieval Mode
- CV_RETR_EXTERNAL gives "outer" contours, so if you have (say) one contour enclosing another (like concentric circles), only the outermost is given. Ex - 0,1,2.
- CV_RETR_LIST gives all the contours and doesn't even bother calculating the hierarchy -- good if you only want the contours and don't care whether one is nested inside another.
- CV_RETR_CCOMP gives contours and organises them into outer and inner contours. Every contour is either the outline of an object, or the outline of an object inside another object (i.e. hole). The hierarchy is adjusted accordingly. This can be useful if (say) you want to find all holes.

<img src="https://docs.opencv.org/master/ccomp_hierarchy.png">

- CV_RETR_TREE calculates the full hierarchy of the contours. So you can say that object1 is nested 4 levels deep within object2 and object3 is also nested 4 levels deep.
<img src="https://docs.opencv.org/master/tree_hierarchy.png">




In [2]:

import cv2
import numpy as np
import time
load_from_disk = True
if load_from_disk:
    hsv_value = np.load('hsv_value.npy')
    
# Define a capture device .Here we use our primary laptop camera(0).If using external usb camera use replace 0 with 1.
cap = cv2.VideoCapture(0)

# Create a window to display the default frame and the threshold frame.
cap.set(3,1280)
cap.set(4,720)

kernel = np.ones((5,5),np.uint8)
canvas = None
x1,y1=0,0
noiseth = 800
while(1):
    _, frame = cap.read()
    frame = cv2.flip( frame, 1 )
    if canvas is None:
        canvas = np.zeros_like(frame)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    if load_from_disk:
        lower_range = hsv_value[0]
        upper_range = hsv_value[1]
    else:           
        lower_range  = np.array([134, 20, 204])
        upper_range = np.array([179, 255, 255])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    mask = cv2.erode(mask,kernel,iterations = 1)
    
# It erodes away the boundaries of foreground object (Always try to keep foreground in white)
# It is normally performed on binary images.
# It needs two inputs, one is our original image, second one is called structuring element or kernel which decides 
#    the nature of operation.
# A pixel in the original image (either 1 or 0) will be considered 1 only if all the pixels under the kernel is 1, 
#    otherwise it is eroded (made to zero).
# Iterations: It is number of times erosion is applied.
    
    mask = cv2.dilate(mask,kernel,iterations = 2)
    
    
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    if contours and cv2.contourArea(max(contours,key = cv2.contourArea)) > noiseth:
        c = max(contours, key = cv2.contourArea)    
        x2,y2,w,h = cv2.boundingRect(c)
        if x1 == 0 and y1 == 0:
            x1,y1= x2,y2     
        else:
            canvas = cv2.line(canvas, (x1,y1),(x2,y2), [255,0,0], 4)
        x1,y1= x2,y2
    else:
        x1,y1 =0,0
    frame = cv2.add(frame,canvas)
    stacked = np.hstack((canvas,frame))
    cv2.imshow('VIRTUAL PEN',cv2.resize(stacked,None,fx=0.6,fy=0.6))

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
    if k == ord('c'):
        canvas = None

cv2.destroyAllWindows()
cap.release()